In [1]:
import tensorflow as tf
from keras.preprocessing import image
from keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
import cv2
import tkinter as tk
from tkinter import filedialog
from keras.models import model_from_json
import json


# Load the emotion detection model

In [2]:
json_file = open(r'C:\Users\harsh\Protype(FER)\Emotion-Model\emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)
emotion_model.load_weights(r"C:\Users\harsh\Protype(FER)\Emotion-Model\emotion_model.h5")
print("Loaded model from disk")


Loaded model from disk


In [3]:

emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}


# GET FILE ()

In [4]:

def get_file_path():
    root = tk.Tk()
    root.withdraw()

    file_path = filedialog.askopenfilename(title="Select a file", filetypes=[("Image and Video files", "*.png;*.jpg;*.jpeg;*.gif;*.mp4")])

    return file_path


# Processing Image Frames

In [5]:

def process_image_frame(frame, face_detector):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    return frame


# Processing Video Frames

In [6]:

def process_video_frame(frame, face_detector):
    frame = cv2.resize(frame, (1280, 720))
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    return frame



# Get The File Path Using The File Dialog


In [7]:
file_path = get_file_path()

# Check if the file is an image or video
if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
    # Image processing
    img = cv2.imread(file_path)
    face_detector = cv2.CascadeClassifier(r'C:\Users\harsh\Protype(FER)\haarcascades\haarcascade_frontalface_default.xml')
    result_frame = process_image_frame(img, face_detector)

    # Display the result for images
    cv2.imshow('Emotion Detection', result_frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    # Video processing
    cap = cv2.VideoCapture(file_path)
    face_detector = cv2.CascadeClassifier(r'C:\Users\harsh\Protype(FER)\haarcascades\haarcascade_frontalface_default.xml')

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        result_frame = process_video_frame(frame, face_detector)

        cv2.imshow('Emotion Detection', result_frame)
        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


1/1 [==============================] - 0s 37ms/step
